<a href="https://colab.research.google.com/github/SPS-L/pyeplan/blob/master/examples/SHS_Planning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrative example for optimizing investment and operation of a single household

## Introduction

This example considers a single household and uses PyEPLAN to decide on the necessary generators that need to be installed to cover the load.

![Single Household](https://raw.githubusercontent.com/SPS-L/pyeplan/master/examples/1_bus.png)



This example is designed to be run on Google Colab. If this is the first time you use Colab, please take a 3-minute break and check out [this video](https://youtu.be/inN8seMm7UI).

To get a quickstart, you can simply click on "Runtime->Run all" and accept the prompt. This will execute the entire notebook and you can read through it like a report.

## Preparing the platform to execute the PyEPLAN software

When using Colab to run the PyEPLAN software, you will need to set up the eplatform using the commands defined below.

In [18]:
#@markdown These commands make sure to remove any files remaining from previous executions.

!rm -rf sample_data
!rm -rf 1_bus

In [19]:
#@markdown  The following command installs SVN and copies the project files from Github to Colab for execution.

!apt-get -qq -y install subversion                                     &> /dev/null
!svn export https://github.com/SPS-L/pyeplan/trunk/examples/1_bus      &> /dev/null

In [20]:
#@markdown The following command installs the necessary optimization solver for pyeplan to use.

!apt-get -y -qq install glpk-utils                    &> /dev/null

In [21]:
#@markdown The following command installs PyEPLAN.

!pip install -q pyeplan==0.4.3                       &> /dev/null

This command is only used during debugging. 

In [22]:
#@markdown This command is only used during debugging.

!pip install git+https://github.com/SPS-L/pyeplan.git            &> /dev/null

## Defining the system data

Input files containing the system data are added to PyEPLAN using csv files. These files contain the load demand for each planning hour and load point, candidate and existing generators, network layout data and load and renewable generation profiles. 

The data input and format of different csv files accepted by PyEPLAN software is defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).  

As a basis, the user should to define the load demand and at least one generation input to be able to utilise the investment and operation modules of PyEPLAN.


### Defining the total load demand 

In [23]:
#@title Total power consumption

import pandas as pd
#pd.read_csv("1_bus/prep_dist.csv")

In [24]:
#@title  { form-width: "100%" }
#@title  { form-width: "70%" }
#@markdown Slide to vary the load demand of the household
# reading the csv file
df = pd.read_csv("1_bus/prep_dist.csv")

maximum_load = 90 #@param {type:"slider", min:0, max:150, step:1}
  
# updating the column value/data
df.loc[0, '0'] = maximum_load
  
# writing into the file
df.to_csv("1_bus/prep_dist.csv", index=False)


### Defining the potential investment candidates

In [25]:
#@title Wind turbines

pd.read_csv("1_bus/cwin_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,186,0,0,0,50,0,0
1,0,186,0,0,0,50,0,0


In [26]:
#@title Solar Photovoltaics
pd.read_csv("1_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,109,0,0,0,50,0,0
1,0,109,0,0,0,50,0,0


In [27]:
#@title Diesel generators
pd.read_csv("1_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12,0.4,0,0,0,0,0,NaN
1,0,12,0.4,0,0,0,0,0,NaN


## Using the invesment and operation planning module from PyEPLAN 

The following command calls on all the PyEPLAN modules to be included in the current Colab platform.

In [28]:
#@markdown The following command calls on all the PyEPLAN modules to be included in the current Colab platform.

import pyeplan as pye

### Initialising the PyEPLAN invesment planning module   

The module is initialised with different inputs including following:
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

The following command is then used to set the different inputs based on the user's preferences.

In [29]:
#@markdown The following command is then used to set the different inputs based on the user's preferences.
inosys = pye.inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

## Determining the optimal investment 

The input arguments that should be provided to the investment planning module include:
* *solver* : program used to calculate the optimal solution to the optimisation problem.

* *onlyopr* : parameter set to 'True' if the user desires only to solve an operation planning problem. Otherwise if the user needs to find the optimal investment i.e., investment planning, the parameter is set to 'False'.

In [30]:
#@title  { vertical-output: true }
#@markdown The following command is used to solve the planning problem and determine the optimal investment candidates for the household.

inosys.solve(solver = 'glpk', onlyopr = False)  

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp9rptlc1j.glpk.raw --wglp /tmp/tmpbw417qn0.glpk.glp --cpxlp
 /tmp/tmptafb3fru.pyomo.lp
Reading problem data from '/tmp/tmptafb3fru.pyomo.lp'...
42 rows, 31 columns, 81 non-zeros
247 lines were read
Writing problem data to '/tmp/tmpbw417qn0.glpk.glp'...
210 lines were written
GLPK Simplex Optimizer, v4.65
42 rows, 31 columns, 81 non-zeros
Preprocessing...
9 rows, 14 columns, 28 non-zeros
Scaling...
 A: min|aij| =  3.333e-01  max|aij| =  4.000e+01  ratio =  1.200e+02
GM: min|aij| =  7.598e-01  max|aij| =  1.316e+00  ratio =  1.732e+00
EQ: min|aij| =  5.774e-01  max|aij| =  1.000e+00  ratio =  1.732e+00
Constructing initial basis...
Size of triangular part is 9
*     0: obj =   3.285000000e+06 inf =   0.000e+00 (4)
*     6: obj =   1.400000000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (56560 bytes)
Writing basic solution to '/tmp/tmp9rptlc1j.glp

## Results

Below we show the capital costs and operational costs that will be incured to satify the electricity load in the household.

 

In [31]:
#@title Total Investment and Operational Costs
inosys.resCost()

,total costs,14000.0
0,total investment costs,14000.0
1,total operation costs,0.0


In [32]:
#@title Number and capacity of wind units installed
#@markdown The capacity and location of wind units installed is:
inosys.resWind()

,Installed Capacity (kW),Bus
Unit,,
1,16.5,0
2,0.0,0


In [33]:
#@title Number and capacity of solar units installed
#@markdown The capacity and location of solar units installed is:

inosys.resSolar()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,50.0,0


In [34]:
#@title Number and capacity of diesel units installed
#@markdown The capacity and location of diesel units installed is:


inosys.resConv()

,Installed Capacity (kW),Bus
Unit,,
1,0.0,0
2,0.0,0


## Exercise
In this exercise, you will vary the load demand using the slider above and observe the changes in the solution provided by PyEPLAN. After every change you make, click on "Runtime->Run all" and accept the prompt. This will execute the entire notebook with the new values. 

Increase and reduce the total load demand using the slider and investigate the changes in the optimal solution i.e.:

* How do the total system costs change with a change in load demand?

* How do the investment candidates and their sizes change with a change in load demand?